## Bloomberg BQuant Webinar Series: <br> Improving Your Portfolio's ESG Score
This is a companion notebook to the "Seeing Green: Improving Your Portfolio's ESG Score" webinar.

In [35]:
import bql
import pandas as pd
from collections import OrderedDict 
import numpy as np
import bqport
import bqviz as bqv
from bqwidgets import DataGrid
import heatmap

In [36]:
bq = bql.Service()

## Create, Save , And Load A Portfolio

In [37]:
req = bql.Request(bq.univ.members('BE500 Index').as_of('2019-10-01'),{'ID':bq.data.ID()})
BE500_df = bq.execute(req)[0].df()
BE500_df.head()

,DATE,Weights,Positions,ORIG_IDS,ID
ID,,,,,
1COV GR Equity,2019-10-01,0.085080,183.000000,BE500 INDEX,1COV GR Equity
8TRA GR Equity,2019-10-01,0.124584,500.000000,BE500 INDEX,8TRA GR Equity
A2A IM Equity,2019-10-01,0.055466,3132.905216,BE500 INDEX,A2A IM Equity
AAK SS Equity,2019-10-01,0.047288,253.730928,BE500 INDEX,AAK SS Equity
AAL LN Equity,2019-10-01,0.279366,1270.522368,BE500 INDEX,AAL LN Equity


In [38]:
np.random.seed(seed=1)
BE500_df['Demo_Port_ Wght'] = np.random.choice([0,1,2],BE500_df.shape[0])
BE500_df['Demo_Port_ Wght'] = BE500_df['Demo_Port_ Wght'] / BE500_df['Demo_Port_ Wght'].sum() * 100
BE500_df.head()

,DATE,Weights,Positions,ORIG_IDS,ID,Demo_Port_ Wght
ID,,,,,,
1COV GR Equity,2019-10-01,0.085080,183.000000,BE500 INDEX,1COV GR Equity,0.205761
8TRA GR Equity,2019-10-01,0.124584,500.000000,BE500 INDEX,8TRA GR Equity,0.000000
A2A IM Equity,2019-10-01,0.055466,3132.905216,BE500 INDEX,A2A IM Equity,0.000000
AAK SS Equity,2019-10-01,0.047288,253.730928,BE500 INDEX,AAK SS Equity,0.205761
AAL LN Equity,2019-10-01,0.279366,1270.522368,BE500 INDEX,AAL LN Equity,0.205761


In [67]:
port_df = BE500_df.loc[:,['DATE','Demo_Port_ Wght']].reset_index() #idd 
port_df.columns =['security','date','weight']
port_df = port_df.set_index(['date','security'])
port_df.drop(port_df[port_df['weight'] == 0].index,inplace=True)
port_df.head()

weight
date       security                
2019-10-01 1COV GR Equity  0.205761
           AAK SS Equity   0.205761
           AAL LN Equity   0.205761
           ABF LN Equity   0.205761
           AC FP Equity    0.205761

In [40]:
portfolios = bqport.list_portfolios()
portfolios[:5]

[{'id': 'PORTFOLIO:31:-11591334:18', 'name': 'jktest3'},
 {'id': 'PORTFOLIO:31:-11591334:19', 'name': 'SWPM deals'},
 {'id': 'PORTFOLIO:31:-11591334:10', 'name': 'JWJ_ETFS'},
 {'id': 'PORTFOLIO:31:-11591334:11', 'name': 'jk6'},
 {'id': 'PORTFOLIO:31:-11591334:12', 'name': 'Upload Test'}]

In [41]:
port_name = "Demo_ESG_Portfolio"

In [42]:
try:
    port = bqport.load_portfolio(name=port_name)
    
except:
    print('Portfoliio not found \n Creating porfolio')
    new_port = bqport.new_portfolio(from_=port_df, type_=bqport.positions.PositionType.FIXED_WEIGHT,name=port_name)
    new_port.save()
    port = bqport.load_portfolio(name=port_name)

print('Portfoliio loaded')

Portfoliio loaded


In [43]:
port.to_dataframe().head()

weight      quantity     price currency  \
date       security                                                     
2019-10-01 UBSG SW Equity   0.411523  36846.653843    11.130      CHF   
           US IM Equity     0.205761  77861.814639     2.421      EUR   
           SMIN LN Equity   0.411523  21536.165596  1563.000      GBp   
           AMS SM Equity    0.205761   2912.600041    64.720      EUR   
           HUH1V FH Equity  0.205761   5175.823451    36.420      EUR   

                             fx_rate          value  
date       security                                  
2019-10-01 UBSG SW Equity   1.003462  411523.011665  
           US IM Equity     1.091550  205760.944386  
           SMIN LN Equity   0.012226  411522.883471  
           AMS SM Equity    1.091550  205760.967774  
           HUH1V FH Equity  1.091550  205760.984600

In [44]:
# test_fld = {'Wgt':bq.data.ID()['weights'].group(bq.data.gics_sector_name()).sum()}
print(port.id)
portfolio = port.id.split(':-')[1].replace(':','-')
print(portfolio)

PORTFOLIO:31:-11591334:56
11591334-56


## Analyse Portfolio And Benchmark ESG Characteristics

In [45]:
esg_factors = OrderedDict()

esg_factors['Tot CO2/Sales'] = bq.data.TOT_GHG_CO2_EM_INTENS_PER_SALES(fpt = 'A',fpo='-1',currency='USD')
esg_factors['Fair ReNum'] = bq.data.FAIR_REMUNERATION_POLICY(fpt = 'A',fpo='-1').replacenonnumeric(replaceval = '0') * 100
esg_factors['Ind Dir'] = bq.data.INDEPENDENT_LEAD_DIRECTOR(fpt = 'A',fpo='-1').replacenonnumeric(replaceval = '0') * 100

In [46]:
def esg_characteristic(factors, univ, grp):
    
    fld = OrderedDict()
    cols = []
    
    wgts = bq.data.ID()['weights'].group(grp)
    
    for key,value in factors.items():
        cols.append(key)
        fld[key] = value.group(grp).wavg(wgts)
    
    fld['Weight'] = wgts.sum()
    
    req = bql.Request(univ,fld)
    resp = bq.execute(req)
    
    dfs =[]
    
    for r in resp:
        dfs.append(r.df())
    
    df = pd.concat(dfs,axis=1)
    cols = ['Weight'] + cols
    df = df.loc[:,cols].round(2)
    
    return df

In [47]:
port_esg = esg_characteristic(esg_factors,port_univ,bq.data.gics_sector_name())
port_esg

,Weight,Tot CO2/Sales,Fair ReNum,Ind Dir
ID,,,,
Communication Services,7.41,28.30,11.11,41.67
Consumer Discretionary,11.93,51.15,5.17,36.21
Consumer Staples,10.29,73.40,14.00,38.00
Energy,3.29,408.70,0.00,37.50
Financials,19.14,5.68,7.53,48.39
Health Care,5.56,34.99,3.70,14.81
Industrials,19.34,80.11,3.19,32.98
Information Technology,4.12,33.82,15.00,15.00
Materials,8.02,955.02,2.56,41.03


In [48]:
bench = bq.univ.members('BE500 Index')
bench_esg = esg_characteristic(esg_factors,bench,bq.data.gics_sector_name())
bench_esg

,Weight,Tot CO2/Sales,Fair ReNum,Ind Dir
ID,,,,
Communication Services,5.26,38.30,6.24,38.52
Consumer Discretionary,10.38,29.83,10.65,34.41
Consumer Staples,14.14,52.93,33.60,59.00
Energy,5.81,317.77,1.08,61.34
Financials,17.51,4.81,6.04,40.79
Health Care,12.96,31.01,17.23,21.25
Industrials,14.47,83.84,2.10,31.54
Information Technology,5.75,23.76,6.77,12.27
Materials,6.16,969.31,3.11,45.62


In [49]:
comp_esg = pd.concat([port_esg.add_prefix('Port '),bench_esg.add_prefix('Bench ')],axis=1)

cols = port_esg.columns
re_ordered_cols = []

for c in cols:
    comp_esg[c+' Diff'] = port_esg[c] - bench_esg[c]
    re_ordered_cols = re_ordered_cols + ['Port '+c,'Bench '+c,c+' Diff'] 

comp_esg = comp_esg[re_ordered_cols]
comp_esg

,Port Weight,Bench Weight,Weight Diff,Port Tot CO2/Sales,Bench Tot CO2/Sales,Tot CO2/Sales Diff,Port Fair ReNum,Bench Fair ReNum,Fair ReNum Diff,Port Ind Dir,Bench Ind Dir,Ind Dir Diff
ID,,,,,,,,,,,,
Communication Services,7.41,5.26,2.15,28.30,38.30,-10.00,11.11,6.24,4.87,41.67,38.52,3.15
Consumer Discretionary,11.93,10.38,1.55,51.15,29.83,21.32,5.17,10.65,-5.48,36.21,34.41,1.80
Consumer Staples,10.29,14.14,-3.85,73.40,52.93,20.47,14.00,33.60,-19.60,38.00,59.00,-21.00
Energy,3.29,5.81,-2.52,408.70,317.77,90.93,0.00,1.08,-1.08,37.50,61.34,-23.84
Financials,19.14,17.51,1.63,5.68,4.81,0.87,7.53,6.04,1.49,48.39,40.79,7.60
Health Care,5.56,12.96,-7.40,34.99,31.01,3.98,3.70,17.23,-13.53,14.81,21.25,-6.44
Industrials,19.34,14.47,4.87,80.11,83.84,-3.73,3.19,2.10,1.09,32.98,31.54,1.44
Information Technology,4.12,5.75,-1.63,33.82,23.76,10.06,15.00,6.77,8.23,15.00,12.27,2.73
Materials,8.02,6.16,1.86,955.02,969.31,-14.29,2.56,3.11,-0.55,41.03,45.62,-4.59


In [50]:
bqv.BarPlot(comp_esg[['Port Tot CO2/Sales','Bench Tot CO2/Sales','Tot CO2/Sales Diff']]).set_style().show()

GridBox(children=(Figure(animation_duration=500, axes=[Axis(color='white', grid_color='#3c3c3c', grid_lines='d…

## Score An Investable Universe

In [51]:
eqy_univ = bq.univ.equitiesuniv(["ACTIVE","PRIMARY"])

In [52]:
mkt_cap = bq.data.cur_mkt_cap(currency='USD') > 1000000000

sector = bq.data.gics_sector_name().len() > 0

west_eur = ['AD','AT','BE','CS','DK','FO','FI','FR','DE','GI','GR','GL','GG',
            'VA','IS','IE','IM','IT','JE','LI','LU','MT','MC','NL','NO','PT','CH','SM','ZB','ES','SE','GB']

country = bq.data.cntry_of_risk().in_(west_eur)

In [53]:
investable_univ = bq.univ.filter(eqy_univ,mkt_cap)
investable_univ = bq.univ.filter(investable_univ,sector)
investable_univ = bq.univ.filter(investable_univ,country)

In [54]:
invest_req = bql.Request(investable_univ,{'count':bq.data.ID().group(bq.data.gics_sector_name()).count()})

In [55]:
bq.execute(invest_req)[0].df()

,ORIG_IDS,GICS_SECTOR_NAME(),count
ID,,,
Communication Services,None,Communication Services,80
Consumer Discretionary,None,Consumer Discretionary,141
Consumer Staples,None,Consumer Staples,83
Energy,None,Energy,38
Financials,None,Financials,211
Health Care,None,Health Care,96
Industrials,None,Industrials,253
Information Technology,None,Information Technology,82
Materials,None,Materials,92


In [56]:
E_factors = OrderedDict()

E_factors['Tot GHG'] = bq.data.TOT_GHG_CO2_EM_INTENS_PER_SALES(fpt = 'A',fpo='-1',currency='USD')
E_factors['Energy Int'] = bq.data.ENERGY_INTENSITY_PER_SALES(fpt = 'A',fpo='-1',currency='USD')
E_factors['Waste Gen'] = bq.data.WASTE_GENERATED_PER_SALES(fpt = 'A',fpo='-1',currency='USD')

In [57]:
def universe_stats(factors,univ,grp):
    flds = OrderedDict()
    
    flds['Count'] = bq.data.ID().group(grp).count()
    
    for key,value in factors.items():
        flds['Cov '+key] = value.group(grp).count() / flds['Count'] * 100
        flds['Avg '+key] = value.group(grp).avg()
        flds['Std '+key] = value.group(grp).std()
        
    req = bql.Request(univ,flds)
    resp = bq.execute(req)
    
    dfs =[]

    for r in resp:
        dfs.append(r.df())

    univ_df = pd.concat(dfs,axis=1)

    univ_df = univ_df.loc[:,[k for k in flds.keys()]]
    
    return univ_df

In [58]:
univ_stats_df = universe_stats(E_factors,investable_univ,bq.data.gics_sector_name())

In [59]:
univ_stats_df.round(2)

,Count,Cov Tot GHG,Avg Tot GHG,Std Tot GHG,Cov Energy Int,Avg Energy Int,Std Energy Int,Cov Waste Gen,Avg Waste Gen,Std Waste Gen
ID,,,,,,,,,,
Communication Services,80,63.75,26.92,35.26,52.50,93.94,100.07,38.75,1.51,3.64
Consumer Discretionary,141,67.38,42.53,79.00,48.94,173.56,305.81,36.88,16.48,26.55
Consumer Staples,83,60.24,75.54,131.12,50.60,273.46,411.95,44.58,23.28,33.03
Energy,38,68.42,649.73,1045.01,55.26,1833.18,1986.41,50.00,23.78,39.11
Financials,211,51.66,5.34,17.68,41.71,24.38,69.69,27.96,7.03,37.87
Health Care,96,41.67,47.00,68.31,35.42,117.59,133.65,34.38,8.14,12.73
Industrials,253,67.19,156.18,595.41,53.36,359.69,1080.74,37.94,306.00,2487.88
Information Technology,82,40.24,21.81,39.88,30.49,47.51,61.88,18.29,1.38,1.65
Materials,92,70.65,894.04,1525.76,65.22,2988.90,2980.45,63.04,1218.17,4344.12


## Score A Portfolio And Benchmark Based On An Investable Universe

In [60]:
def port_bench_comp(port,bench,factors,grp,univ_stats):
    
    univs = OrderedDict()
    univs = {'Port':port,'Bench':bench}
    
    port_bench =[]
    
    for univ_name,univ in univs.items():
        
        flds = OrderedDict()
        wgts = bq.data.ID()['weights'].group(grp)
        
        for name, factor in factors.items():
            flds[name] = factor.group(grp).wavg(wgts)
            
        req = bql.Request(univ,flds)
        resp = bq.execute(req)

        dfs =[]

        for r in resp:
            dfs.append(r.df())

        univ_df = pd.concat(dfs,axis=1)
        univ_df = univ_df.loc[:,[k for k in factors.keys()]]
        
        for c in univ_df.columns:
            univ_df[c] = (univ_df[c] - univ_stats["Avg "+c]) / univ_stats["Std "+c]
        
        univ_df['Avg'] = univ_df.mean(axis=1) 
        univ_df = univ_df.add_prefix(univ_name+" ")
        
        port_bench.append(univ_df)
        
    comp_df = pd.concat(port_bench,axis=1)
    
    re_ordered_cols=[]
   
    for k in factors.keys():
            re_ordered_cols.append('Port ' + k)
            re_ordered_cols.append('Bench ' + k)
    
    re_ordered_cols.append('Port Avg')
    re_ordered_cols.append('Bench Avg')
    
    return comp_df[re_ordered_cols]
    

In [61]:
comp_score = port_bench_comp(port_univ,bench,E_factors,bq.data.gics_sector_name(),univ_stats_df)
comp_score.sort_values('Port Avg')

,Port Tot GHG,Bench Tot GHG,Port Energy Int,Bench Energy Int,Port Waste Gen,Bench Waste Gen,Port Avg,Bench Avg
ID,,,,,,,,
Utilities,-0.208440,0.124125,-0.169600,0.224074,-0.272187,-0.156856,-0.216742,0.063781
Energy,-0.230647,-0.317666,-0.257044,-0.355982,-0.144687,-0.208586,-0.210793,-0.294078
Consumer Staples,-0.016273,-0.172423,-0.246688,-0.110067,-0.208710,-0.076594,-0.157224,-0.119694
Industrials,-0.127753,-0.121496,-0.162996,-0.179709,-0.111824,-0.109766,-0.134191,-0.136990
Communication Services,0.039162,0.322883,-0.142556,0.228755,-0.255989,-0.207513,-0.119794,0.114708
Consumer Discretionary,0.109001,-0.160839,0.004828,-0.220195,-0.319787,-0.415612,-0.068653,-0.265548
Health Care,-0.175834,-0.234080,0.094121,-0.194249,0.058579,-0.158299,-0.007711,-0.195543
Financials,0.019423,-0.030034,-0.005610,-0.115278,0.014554,-0.133906,0.009456,-0.093073
Materials,0.039970,0.049333,0.076888,-0.064969,0.004596,0.857592,0.040485,0.280652


In [62]:
heatmap.plot_heat_map(comp_score)

Figure(axes=[Axis(grid_lines='none', scale=OrdinalScale(), side='top'), Axis(grid_lines='none', orientation='v…

## Screen For Securities That Would Improve The Porfolio's ESG Score

In [63]:
sector = bq.data.gics_sector_name() == 'Real Estate'

sector_univ = bq.univ.filter(eqy_univ,mkt_cap)
sector_univ = bq.univ.filter(sector_univ,sector)
sector_univ = bq.univ.filter(sector_univ,country)

In [64]:
flds = E_factors
flds['e_score'] = ((bq.data.TOT_GHG_CO2_EM_INTENS_PER_SALES(fpt = 'A',fpo='-1',currency='USD').groupzscore(bq.data.gics_sector_name()) / 3) + 
                    (bq.data.ENERGY_INTENSITY_PER_SALES(fpt = 'A',fpo='-1',currency='USD').groupzscore(bq.data.gics_sector_name()) / 3) +
                    (bq.data.WASTE_GENERATED_PER_SALES(fpt = 'A',fpo='-1',currency='USD').groupzscore(bq.data.gics_sector_name())) / 3) 

In [65]:
req = bql.Request(sector_univ,flds)
resp = bq.execute(req)

dfs =[]

for r in resp:
    dfs.append(r.df())
    
sector_df = pd.concat(dfs,axis=1)
sector_df = sector_df.loc[:,[k for k in flds.keys()]]

In [66]:
format_df = sector_df.dropna().sort_values('e_score').reset_index().round(2)
cols = format_df.columns
dgrid_col_width = 100

DataGrid(data=format_df,
          column_defs = [{'headerName': 'Security', 'field': cols[0], 'width': 200},
                         {'headerName': cols[1], 'field' : cols[1], 'width' : dgrid_col_width},
                         {'headerName': cols[2], 'field' : cols[2], 'width' : dgrid_col_width},
                         {'headerName': cols[3], 'field' : cols[3], 'width' : dgrid_col_width},
                         {'headerName': cols[4], 'field' : cols[4], 'width' : dgrid_col_width}],
         layout = {'width':'650px','height':'500px'})


DataGrid(column_defs=[{'width': 200, 'field': 'ID', 'headerName': 'Security'}, {'width': 100, 'field': 'Tot GH…